In [1]:
include("main.jl")

#println(var_values)

#--------------------------------------------------------------------------------
#                                                                                
#                                  Bilevel DNEP                                   
#                                                                                
#                                Graduation work                                 
#                                                                                
#--------------------------------------------------------------------------------
# @ Manon Cornet

Running a simulation with the following characteristics:
--------------------------------------------------------

╭─────────────┬───────┬───────┬─────────┬─────────┬─────────┬─────────┬─────────┬───────────╮
│  Model Type │    EV │   EHP │ Delta_t │ PV_CAPA │     EIC │     EEC │   DSOEC │       GCC │
│         [-] │   [-] │   [-] │   [min] │   [MVA] │ [€/kWh] │ [€/kWh] │ [€/kWh] │ [€/kVA/y] │
├─────────

#--------------------------------------------------------------------------------


hello


hello


1


...

Matrix{

Float64}
hello
Matrix{Float64}


┌ Info: Building model...
└ @ Main.UpperLevel /Users/manoncornet/Documents/University/TFE/Bilevel_DNEP/src/UpperLevel/build_model.jl:16
┌ Info: Built model in 6.84 seconds
└ @ Main.UpperLevel /Users/manoncornet/Documents/University/TFE/Bilevel_DNEP/src/UpperLevel/build_model.jl:59


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1


A JuMP Model
Minimization problem with:
Variables: 41
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 12 constraints
`JuMP.AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 17 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 27 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.SecondOrderCone`: 2 constraints
`Vector{JuMP.AffExpr}`-in-`MathOptInterface.RotatedSecondOrderCone`: 3 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 14 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`JuMP.VariableRef`-in-`MathOptInterface.ZeroOne`: 11 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: Alpha, Beta, DSO_costs, I_sqr_k, P_ij_k, P_sub, Q_ij_k, Q_sub, S_sub, S_sub_capa, User_costs, V_sqr, Y, delta_t, nb_sign_days, network_data, network_topology, time_steps

In [2]:
print(network.pu_basis)
print(network.conductors)
print(network.buses)

(base_power = 1.0, base_voltage = 34.5, base_current = 0.028985507246376812, base_impedance = 1190.25)Main.UpperLevel.Conductor

[Main.UpperLevel.Conductor("Poppy", 0.00046225582860743544, 0.0003604284814114682, 7.9350000000000005, 10.0), Main.UpperLevel.Conductor("Oxlip", 0.0002307918504515858, 0.0003377441713925646, 11.73, 12.0)]

Main.UpperLevel.Bus[Main.UpperLevel.Substation(Main.UpperLevel.Node(1, (x = 0.0, y = 0.1)), false, (V_min = 0.95, V_max = 1.05), 15.0, 0.0, nothing, nothing, nothing), Main.UpperLevel.Substation(Main.UpperLevel.Node(2, (x = 0.0, y = 0.0)), false, (V_min = 0.95, V_max = 1.05), 15.0, 0.0, nothing, nothing, nothing), Main.UpperLevel.User(Main.UpperLevel.Node(3, (x = 0.1, y = 0.1)), (V_min = 0.95, V_max = 1.05), 0.4, nothing, Main

.UpperLevel.Profile([0.09768183630460787, 0.24599874693854307, 0.44056501680241505, 0.48840918152303925, 0.48840918152303936, 0.48840918152303925, 0.5203052913367888, 0.4912000911317424, 0.6299481688215527, 0.6538702511818648, 0.6299481688215529, 0.6143988152873499, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.45651307170928984, 0.5238936036908357, 0.793415731617019, 0.793415731617019, 0.5781169903742098, 0.36720396423079116, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.16944808338554426, 0.45651307170928984, 0.45651307170928984, 0.45651307170928984, 0.45651307170928984, 0.16944808338554426, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.09768183630460787, 0.16506236828615373, 0.434584496212337, 0.434584496212337, 0.578116

, 1.9655977672723133, 1.5525431451842573, 1.172182035655294, 1.7782081221165347, 1.9480549068747506, 2.1410263712479356, 2.292532892863246, 2.4280913595716807, 2.240701714415903, 1.9217406162784076, 1.8021302044768472, 1.8228626758557842, 1.8774847639118302, 1.7642535740730196, 3.8925215013954553, 3.474682462835337, 2.5058381272426953, 2.3822407017144163, 2.350344591900667, 4.380531981545822, 1.694879535228114, 1.7104288887623171, 1.5900210742154126, 1.6486301759981774, 1.6972717434641453, 1.6605912171783335, 2.226747166372387, 1.7084353818989577, 1.7084353818989577, 2.064077006322265, 2.250669248732699, 2.226747166372387, 2.216380930682919, 2.086802984564562, 1.4791820926126333, 2.738281027510394, 0.9229936777353763, 1.0505781169903745, 1.2431508799908868, 1.7000626530728487, 2.1410263712479356, 2.082815970837843, 2.125078316341061, 2.107535455943498, 1.519450931252492, 1.6406561485447404, 1.604773025004272, 1.5469613259668509, 1.8204704676197532, 2.282565358546449, 2.1737198838070286

 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.10166885003132656, 0.09568832944124851, 0.09568832944124851, 0.09568832944124851, 0.09568832944124851, 0.09568832944124851, 0.20094549182662186, 0.35883123540468187, 0.35883123540468187, 0.35883123540468187, 0.14831691063393518, 0.09568832944124851, 0.0956883294

In [3]:
using JuMP
latex_formulation(upper_model)
# Problem is that etiher P_ij or Q_ij_k

$$ \begin{aligned}
\min\quad & 0.02 Alpha_{1,1} + 0.024000000000000004 Alpha_{1,2} + 0.02828427124746191 Alpha_{2,1} + 0.033941125496954286 Alpha_{2,2} + 0.02 Alpha_{3,1} + 0.024000000000000004 Alpha_{3,2} + 20 S\_sub\_capa_{1} + 20 S\_sub\_capa_{2} + 0.00885797731568998 I\_sqr\_k_{1,1,1} + 0.004422548834278513 I\_sqr\_k_{1,1,2} + 0.012527071655041994 I\_sqr\_k_{1,2,1} + 0.006254428541693995 I\_sqr\_k_{1,2,2} + 0.00885797731568998 I\_sqr\_k_{1,3,1} + 0.004422548834278513 I\_sqr\_k_{1,3,2}\\
\text{Subject to} \quad & Alpha_{1,1} + Alpha_{1,2} - Y_{1} = 0\\
 & Alpha_{2,1} + Alpha_{2,2} - Y_{2} = 0\\
 & Alpha_{3,1} + Alpha_{3,2} - Y_{3} = 0\\
 & Y_{1} + Y_{2} + Y_{3} = 2\\
 & -P\_sub_{1,1} + P\_ij\_k_{1,1,1} + P\_ij\_k_{1,1,2} = 0\\
 & -P\_sub_{1,2} + P\_ij\_k_{1,2,1} + P\_ij\_k_{1,2,2} = 0\\
 & -Q\_sub_{1,1} + Q\_ij\_k_{1,1,1} + Q\_ij\_k_{1,1,2} = 0\\
 & -Q\_sub_{1,2} + Q\_ij\_k_{1,2,1} + Q\_ij\_k_{1,2,2} = 0\\
 & -P\_ij\_k_{1,1,1} - P\_ij\_k_{1,2,1} + P\_ij\_k_{1,3,1} - P\_ij\_k_{1,1,2}

In [4]:
objective, time = UpperLevel.solve_model(upper_model, formulation.powerflow)
save_struct(network_topology, "network_topology.json")
save_struct(network, "network_data.json")
# 1. What to do is to update the structures when going through this function
# 2. 


Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 68 rows, 53 columns and 221 nonzeros
Model fingerprint: 0x8f2e82b3
Model has 5 quadratic constraints
Variable types: 42 continuous, 11 integer (11 binary)
Coefficient statistics:
  Matrix range     [2e-09, 1e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [4e-03, 2e+01]
  Bounds range     [9e-01, 1e+00]
  RHS range        [4e-02, 2e+00]
Presolve removed 23 rows and 11 columns
Presolve time: 0.01s
Presolved: 45 rows, 42 columns, 165 nonzeros
Presolved model has 5 quadratic constraint(s)
Variable types: 36 continuous, 6 integer (6 binary)

Root relaxation: objective 4.000000e-02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      | 

In [ ]:
# Represent graphically the model 

function print_network(network_data::Network, network_topology::N)

end